In [1]:
import cv2
import mediapipe as mp
import time
from plyer import notification
import os

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [3]:
neck_forward = False
neck_left = False
slouch = False
neck_down = False

In [4]:
time_elapsed_neck_forward = 0
time_elapsed_neck_left = 0
time_elapsed_slouch = 0
time_elapsed_neck_down = 0
margin_time = 5

In [5]:
time_notified_neck_forward = 0
time_notified_neck_left = 0
time_notified_slouch = 0
time_notified_neck_down = 0

In [14]:
cap = cv2.VideoCapture(0)
cap.set(3, 1280)
cap.set(4, 720)
# Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.8, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Drawing rectangle on top for message
        cv2.rectangle(image, (0, 0), (1280, 90), (255, 255, 0), -1)

        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            print(landmarks[mp_pose.PoseLandmark.LEFT_EYE_OUTER.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x)

            left_ear_x = landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x
            left_eye_outer_x = landmarks[mp_pose.PoseLandmark.LEFT_EYE_OUTER.value].x
            left_shoulder_x = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x
            left_shoulder_y = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y
            mouth_y = landmarks[mp_pose.PoseLandmark.MOUTH_LEFT.value].y

            # For neck forward and right tilt
            if left_shoulder_x - left_eye_outer_x >= 0.325:
                print("For neck forward and right tilt")
                if not neck_forward:
                    start_time_neck_forward = time.time()
                    neck_forward = True
                else:
                    time_elapsed_neck_forward = time.time() - start_time_neck_forward
                cv2.putText(image, "Your neck is forward!", (0, 85), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 0), 6, cv2.LINE_AA)

            # For left neck tilt
            elif left_shoulder_x - left_eye_outer_x <= 0.235:
                print("For left neck tilt")
                if not neck_left:
                    start_time_neck_left = time.time()
                    neck_left = True
                else:
                    time_elapsed_neck_left = time.time() - start_time_neck_left
                cv2.putText(image, "Your neck is tilted to left!", (0, 85), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 0), 6, cv2.LINE_AA)

            # For Slouching down
            elif left_shoulder_y >= 0.89:
                print("For Slouching down")
                if not slouch:
                    start_time_slouch = time.time()
                    slouch = True
                else:
                    time_elapsed_slouch = time.time() - start_time_slouch
                cv2.putText(image, "You are slouching down!", (0, 85), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 0), 6, cv2.LINE_AA)

            # For neck down
            elif left_shoulder_y - mouth_y <= 0.25:
                print("For neck down")
                if not neck_down:
                    start_time_neck_down = time.time()
                    neck_down = True
                else:
                    time_elapsed_neck_down = time.time() - start_time_neck_down
                cv2.putText(image, "Your neck is down!", (0, 85), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 0), 6, cv2.LINE_AA)

            # When nothing is wrong
            else:
                print("When nothing is wrong")
                neck_forward = False
                neck_left = False
                slouch = False
                neck_down = False

                time_elapsed_neck_forward = 0
                time_elapsed_neck_left = 0
                time_elapsed_slouch = 0
                time_elapsed_neck_down = 0
                cv2.putText(image, "Posture is perfect!", (0, 85), cv2.FONT_HERSHEY_PLAIN, 5, (0, 0, 0), 6, cv2.LINE_AA)

            # Loop For notification and sound
            if time_elapsed_neck_forward >= margin_time:
                if (time.time() - time_notified_neck_forward)>10:
                    time_notified_neck_forward = time.time()
                    notification.notify('Wrong Posture!!', "Your neck is forward!", timeout=10)
                    os.startfile("clock-alarm-8761.mp3")


            elif time_elapsed_neck_left >= margin_time:
                if (time.time() - time_notified_neck_left)>10:
                    time_notified_neck_left = time.time()
                    notification.notify('Wrong Posture!!', "Your neck is tilted to left!", timeout=10)
                    os.startfile("clock-alarm-8761.mp3")

            elif time_elapsed_slouch >=  margin_time:
                if (time.time() - time_notified_slouch)>10:
                    time_notified_slouch = time.time()
                    notification.notify('Wrong Posture!!', "You are slouching down!", timeout=10)
                    os.startfile("clock-alarm-8761.mp3")

            elif time_elapsed_neck_down >= margin_time:
                if (time.time() - time_notified_neck_down)>10:
                    time_notified_neck_down = time.time()
                    notification.notify('Wrong Posture!!', "Your neck is down!", timeout=10)
                    os.startfile("clock-alarm-8761.mp3")

        except BaseException:
            pass

        # Render detections
        mp_drawing.draw_landmarks(
            image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(
                color=(
                    245,
                    117,
                    66),
                thickness=2,
                circle_radius=2),
            mp_drawing.DrawingSpec(
                color=(
                    245,
                    66,
                    230),
                thickness=2,
                circle_radius=2))

        # image = cv2.flip(image, 1)
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

0.5650482177734375 0.7952366471290588
For left neck tilt
0.5646001100540161 0.8008142709732056
When nothing is wrong
0.5641316771507263 0.8075550198554993
When nothing is wrong
0.562956690788269 0.8103120923042297
When nothing is wrong
0.5639244318008423 0.8131566047668457
When nothing is wrong
0.5638622641563416 0.8127496838569641
When nothing is wrong
0.5632055401802063 0.8125647306442261
When nothing is wrong
0.5629153251647949 0.8111327886581421
When nothing is wrong
0.5630192160606384 0.8108776807785034
When nothing is wrong
0.5656219124794006 0.8106239438056946
When nothing is wrong
0.5669552087783813 0.809846043586731
When nothing is wrong
0.5671311616897583 0.8087900280952454
When nothing is wrong
0.5680535435676575 0.8084108233451843
When nothing is wrong
0.5676608085632324 0.8082892298698425
When nothing is wrong
0.5676964521408081 0.8082066774368286
When nothing is wrong
0.5676487684249878 0.8047555685043335
When nothing is wrong
0.5678693056106567 0.8044688105583191
When no